In [2]:
from socket import *
from time import ctime
import struct
from threading import Thread
import _thread 
from imutils import face_utils
import playsound
import threading
import time
import logging
import os
import dlib
from dlib import rectangle
import imutils
import cv2
from mtcnn.mtcnn import MTCNN as model
from imutils.video import VideoStream
from threading import Thread
from scipy.spatial import distance as dist
import numpy as np
from scipy.ndimage.filters import gaussian_filter
import pygame
from GradientIntersect import GradientIntersect
from WebcamVideoStream import WebcamVideoStream

speed = 0
system_Activation = 0
statue_Activation = [1,1,1]
capture = WebcamVideoStream(0).start()#cv2.VideoCapture(1)
detector = model()
start = time.time()
i=0
counter=0    
ALARM_ON = False
COUNTER = 0
font = cv2.FONT_HERSHEY_SIMPLEX
t = False
start2 = 0
start1 = 0
left = 0
right = 0
up = 0
down = 0
ea = []
ale = 0
count_pupil = 0
locx_prev = -10
locy_prev = -10
locx_prev1 = -10
locy_prev1 = -10
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]  
gi = GradientIntersect(25,50)
count_eye = 0
eye_right_prev = []
eye_right_curent = []
eye_left_prev = []
eye_left_curent = []
shape1 = []
e_l = []
e_r = []
# end
left1 = 0
left1_prev = time.time()
right1 = 0
count_pupil = 0
right1_prev = time.time()
COUNTER_prev = time.time()
eye_x_prev = time.time()
time_count_pupil_prev = time.time()
count_pupil_faild = 0
eye_x = 0
down1 = 0
up1 = 0
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 40
cascPath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

def contect_with_raspberry():
    global speed,system_Activation,statue_Activation    
    ctrCmd = ["b'Up'","b'Down'"]

    HOST = '192.168.43.88'
    PORT = 21567
    BUFSIZE = 1024
    ADDR = (HOST,PORT)

    tcpSerSock = socket(AF_INET, SOCK_STREAM)
    tcpSerSock.bind(ADDR)
    tcpSerSock.listen(5)

    i = 0
    while True:
        #print ('Waiting for connection')
        tcpCliSock,addr = tcpSerSock.accept()
        #print ('...connected from :', addr)
        try:            
            while True:
                data = ''                       
                data = (tcpCliSock.recv(BUFSIZE))
                data1 = str(data)                                            
                if not data:                
                    break
                else:
                    data = int(data1[2:len(data1)-1])
                    if i >= 1 and i <= 3:
                        statue_Activation[i-1] = data
                    speed = data
                    print('speed : ',data)                            
                    if str(data) == ctrCmd[0]:
                        #Servomotor.ServoUp()
                        #print 'Increase: ',Servomotor.cur_X
                        print('UP')
                    if str(data) == ctrCmd[1]:
                       # Servomotor.ServoDown()
                        #print 'Decrease: ',Servomotor.cur_X
                        print('DOWN')
            if i < 4:
                i +=1

        except KeyboardInterrupt:
            Servomotor.close()
    tcpSerSock.close();

def sound_alarm(path):    
    pygame.mixer.init()
    pygame.mixer.music.load(path)
    pygame.mixer.music.play()

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5]) # ||p2-p6||
    B = dist.euclidean(eye[2], eye[4]) # ||p3-p5||

    C = dist.euclidean(eye[0], eye[3]) # ||p1-p4||

    ear = (A + B) / (2.0 * C) # EAR = (||p2-p6||+||p3-p5||)/(2*||p1-p4||)

    return ear

def detectHC(inputFrame):

    faces = faceCascade.detectMultiScale(
        inputFrame,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    return faces


def eyedetector(gray,rect,frame,COUNTER,ea,COUNTER_prev): 
    global ALARM_ON
         
    shape = predictor(gray, rect)    
    shape = face_utils.shape_to_np(shape)

    cv2.drawContours(frame, [shape], -1, (0, 0, 255), 1)
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)
    ear = (leftEAR + rightEAR) / 2.0
    leftEyeHull = cv2.convexHull(leftEye)
    rightEyeHull = cv2.convexHull(rightEye)
    if ear < EYE_AR_THRESH:
        ea.append(ear)
        COUNTER = (time.time() - COUNTER_prev)
        if (speed<= 60 and COUNTER >= 3) or (speed > 60 and speed < 100 and COUNTER >= 2) or (speed >= 100 and COUNTER >= 1):#EYE_AR_CONSEC_FRAMES:            
            if not ALARM_ON:
                ALARM_ON = True
                if "alarm.wav" != "":                    
                    s=0
                    for i in ea:
                        s +=i                    
                    if s <= len(ea)*0.2:#8:
                        print(COUNTER)
                        t = Thread(target=sound_alarm("alarm.wav"))#,args="alarm.wav")
                        t.deamon = True
                        t.start()
                        print("sleep detection ...............................")                       
                if s <= len(ea)*0.2:#8:
                    print(True)
                    cv2.putText(frame, "DROWSINESS ALERT!", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                ea.clear()
                #COUNTER = 0
                COUNTER_prev = time.time()
                ALARM_ON = False
    else:
        ea.clear()        
        COUNTER_prev = time.time()
        ALARM_ON = False
    cv2.putText(frame, "EAR: {:.2f}".format(ear), (50, 50),
        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)    
    return COUNTER,frame,ea,shape,COUNTER_prev


def draw_roi(image,results,grayFrame,COUNTER):
    global ALARM_ON,ea
    
    for result in results:
        bounding_box = result['box']
        keypoints = result['keypoints']        
        rect = rectangle(bounding_box[0], bounding_box[1],bounding_box[0]+bounding_box[2],bounding_box[1] + bounding_box[3])
        
        cv2.rectangle(image,
                    (bounding_box[0], bounding_box[1]),
                    (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                    (0,155,255),
                    2)
        
        COUNTER,image,ea = eyedetector(grayFrame,rect,image,COUNTER,ea)

        cv2.circle(image,(keypoints['left_eye']), 2, (0,155,255), 2)
        cv2.circle(image,(keypoints['right_eye']), 2, (0,155,255), 2)
        cv2.circle(image,(keypoints['nose']), 2, (0,155,255), 2)
        cv2.circle(image,(keypoints['mouth_left']), 2, (0,155,255), 2)
        cv2.circle(image,(keypoints['mouth_right']), 2, (0,155,255), 2)        
        if keypoints['nose'][0]-keypoints['left_eye'][0] > 3*(keypoints['right_eye'][0]-keypoints['nose'][0]):
            cv2.putText(image,'left',(50, 50), font, 2,(0,0,255),5)
        if 3*(keypoints['nose'][0]-keypoints['left_eye'][0]) < (keypoints['right_eye'][0]-keypoints['nose'][0]):
            cv2.putText(image,'right',(50, 50), font, 2,(0,0,255),5)
            
        return COUNTER,image
    
def drawRectangle(inputFrame, face):    
    outputFrame = cv2.rectangle(inputFrame, (face[0], face[1]), (face[0]+face[2], face[1]+face[3]), (255, 0, 0), 3)
    return outputFrame
        
    
def detection_eyes(self,shape):
    leftEye = shape[self.lStart:self.lEnd]
    rightEye = shape[self.rStart:self.rEnd]
    leftEAR = self.eye_aspect_ratio(leftEye)
    rightEAR = self.eye_aspect_ratio(rightEye)
    #ear = (leftEAR + rightEAR) / 2.0
    leftEyeHull = cv2.convexHull(leftEye)
    rightEyeHull = cv2.convexHull(rightEye)
    x = leftEye[0][0]
    x1 = leftEye[0][0]+(leftEye[3][0] - leftEye[0][0])
    y =  leftEye[1][1]
    y1 = leftEye[1][1]+(leftEye[4][1] - leftEye[1][1])
    x2 = rightEye[0][0]
    x3 = rightEye[0][0]+(leftEye[3][0] - leftEye[0][0])
    y2 =  rightEye[1][1]
    y3 = rightEye[1][1]+(leftEye[4][1] - leftEye[1][1])    
    return y,y1,x,x1,y2,y3,x2,x3

def detect_eye_pupil(time_count_pupil_prev,count_pupil,count_pupil_faild,locx_prev):
    global shape1,frame1,gi    
    while True:        
        if len(shape1)>0  :       
            leftEye = shape1[lStart:lEnd]
            rightEye = shape1[rStart:rEnd]
            x = leftEye[0][0]
            x1 = leftEye[0][0]+(leftEye[3][0] - leftEye[0][0])
            y =  leftEye[1][1]
            y1 = leftEye[1][1]+(leftEye[4][1] - leftEye[1][1])
            x2 = rightEye[0][0]
            x3 = rightEye[0][0]+(leftEye[3][0] - leftEye[0][0])
            y2 =  rightEye[1][1]
            y3 = rightEye[1][1]+(leftEye[4][1] - leftEye[1][1])    

            eye = frame1[y:y1,x:x1]
            eye2 = frame1[y2:y3,x2:x3]

            try:
                eye = cv2.resize(eye,(50,25))
                eye2 = cv2.resize(eye2,(50,25))
            except :
                continue
            gray_eye1 = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
            #gray_eye2 = cv2.cvtColor(eye2, cv2.COLOR_BGR2GRAY)
            loc = gi.locate(gray_eye1, accuracy = 10)
           # loc1 = gi.locate(gray_eye2, accuracy = 10)
            gray_eye1[loc[0],loc[1]] = 255
            #gray_eye2[loc1[0],loc1[1]] = 255
            #print(loc[0],loc[1],loc1[0],loc1[1])
            if locx_prev != -10:
                if ((loc[0] <= locx_prev+5 and loc[0] >= locx_prev-5) and (loc[1] <= locy_prev+5 and loc[1] >= locy_prev-5)): #or ((loc1[0] <= locx_prev1+5 and loc1[0] >= locx_prev1-5) and (loc1[1] <= locy_prev1+5 and loc1[1] >= locy_prev1-5)):
                    count_pupil += 1
                else:
                    #print(count)
                    count_pupil_faild += 1
            locx_prev = loc[0]
            locy_prev = loc[1]
            #locx_prev1 = loc1[0]
            #locy_prev1 = loc1[1]
            time_count_pupil = time.time() - time_count_pupil_prev
            if time_count_pupil >= 6 :                    
                if 10*count_pupil/100 >= count_pupil_faild and count_pupil_faild != 0:
                    t = Thread(target=sound_alarm("other.mp3"))#,args="alarm.wav")
                    t.deamon = True
                    t.start()
                print(count_pupil,count_pupil_faild)
                count_pupil = 0
                count_pupil_faild = 0
                time_count_pupil_prev = time.time() 

            #cv2.rectangle(gray, (xx, yy), (xx, yy), (255, 0, 0), 2)
            cv2.imshow("eyee",gray_eye1)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break         
            #bool_pupil = 0


_thread.start_new_thread(contect_with_raspberry,())
_thread.start_new_thread(detect_eye_pupil,(time_count_pupil_prev,count_pupil,count_pupil_faild,locx_prev))
t = True 
while True:
    #has_frame, frame = capture.read()
    if speed >= system_Activation:
        frame = capture.read()        
        i = i+1    
        counter+=1
        #if i%3 != 0:
         #   continue
        frame = imutils.resize(frame, width=200)
        #assert has_frame, 'No frame'   
        grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if start2-start1 > 1 :
            t=False            
        if t==False and (speed >= 40) :       
            t = True
            start1 = time.time()
            results = detector.detect_faces(frame_RGB)
    #        COUNTER,frame = draw_roi(frame, results,grayFrame,COUNTER)
            for result in results:
                bounding_box = result['box']
                keypoints = result['keypoints']        
                rect = rectangle(bounding_box[0], bounding_box[1],bounding_box[0]+bounding_box[2],bounding_box[1] + bounding_box[3])

                cv2.rectangle(frame,
                            (bounding_box[0], bounding_box[1]),
                            (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                            (0,155,255),
                            2)

                COUNTER,frame,ea,shape,COUNTER_prev = eyedetector(grayFrame,rect,frame,COUNTER,ea,COUNTER_prev)

                cv2.circle(frame,(keypoints['left_eye']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['right_eye']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['nose']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['mouth_left']), 2, (0,155,255), 2)
                cv2.circle(frame,(keypoints['mouth_right']), 2, (0,155,255), 2)
                #print(keypoints['left_eye'][0],keypoints['nose'][0],keypoints['right_eye'][0])                
                if keypoints['nose'][0]-keypoints['left_eye'][0] > 3*(keypoints['right_eye'][0]-keypoints['nose'][0]):
                    cv2.putText(frame,'left',(50, 50), font, 2,(0,0,255),5)
                    left1 =(time.time() - left1_prev)
                else:
                    left1_prev = time.time()
                if 3*(keypoints['nose'][0]-keypoints['left_eye'][0]) < (keypoints['right_eye'][0]-keypoints['nose'][0]):
                    cv2.putText(frame,'right',(50, 50), font, 2,(0,0,255),5)
                    right1 = (time.time()-right1_prev)
                else:
                    right1_prev = time.time()

                if (keypoints['nose'][1]-keypoints['right_eye'][1]) * 2 < (keypoints['mouth_right'][1]-keypoints['nose'][1]):
                    cv2.putText(frame,'up',(50, 50), font, 2,(0,0,255),5)
                    up1 +=1
                else:
                    up1 = 0

                if (keypoints['mouth_right'][1]-keypoints['nose'][1]) * 2 < (keypoints['nose'][1]-keypoints['right_eye'][1]):
                    cv2.putText(frame,'down',(50, 50), font, 2,(0,0,255),5)
                    down1 +=1
                else:
                    down1 = 0

                if (speed >= 0 and speed < 60 and ((left1 >= 3) or (right1 >= 3))) or (speed >= 60 and speed < 100 and ((left1 >= 2) or (right1 >= 2))):#or up1 == 2 or down1 == 2:
                    t = Thread(target=sound_alarm("head.mp3"))#,args="alarm.wav")                    
                    t.deamon = True                    
                    t.start()
                    print("head Orientation ...............................")
                    left1 = 0
                    right1 = 0
                    up1 = 0
                    down1 = 0
                    print('abd')
               # print(keypoints)
               # cv2.imshow('Video', frame)
               # cv2.waitKey(0)
        else :                
            start2 = time.time()
            faces = detectHC(grayFrame)
            if len(faces)>0:
                detectedFace = drawRectangle(frame, faces[0])
                rect = rectangle(faces[0][0], faces[0][1],faces[0][0]+faces[0][2],faces[0][1] + faces[0][3])
                COUNTER,detectedFace,ea,shape,COUNTER_prev = eyedetector(grayFrame,rect,detectedFace,COUNTER,ea,COUNTER_prev)
                frame1 = frame
                shape1 = shape
            else:
                shape1 = []


        cv2.imshow('Video'+str(type), frame)

        # Break while loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release capture
# capture.release()
cv2.destroyAllWindows()
end = time.time()
print(end-start)
print(counter/(end-start))



speed :  0
speed :  0
speed :  0
speed :  0
speed :  0


Unhandled exception in thread started by <function contect_with_raspberry at 0x00000192DD0BF6A8>


speed :  0


OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted

speed :  0
speed :  0
speed :  0
71 22
speed :  0
speed :  0
speed :  0
speed :  0
speed :  0
speed :  0
142 45
speed :  0
speed :  0
14.069685935974121
26.581972170660677
speed :  0
speed :  0
speed :  0
speed :  0
speed :  0
274 5
